#NOT WORKING STUFF



Installing dependencies

In [ ]:
!pip install tensorflow
!pip install scikit.learn
!pip install pandas
!pip install numpy

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

In [ ]:
def convert(imgf, labelf, outf, n):
    f = open(imgf, "rb")
    o = open(outf, "w")
    l = open(labelf, "rb")

    f.read(16)
    l.read(8)
    images = []

    for i in range(n):
        image = [ord(l.read(1))]
        for j in range(28*28):
            image.append(ord(f.read(1)))
        images.append(image)

    for image in images:
        o.write(",".join(str(pix) for pix in image)+"\n")
    f.close()
    o.close()
    l.close()

convert("train-images-idx3-ubyte", "train-labels-idx1-ubyte",
"mnist_train.csv", 60000)
convert("t10k-images-idx3-ubyte", "t10k-labels-idx1-ubyte",
"mnist_test.csv", 10000)

In [ ]:
df_orig_train = pd.read_csv('mnist_train.csv')
df_orig_test = pd.read_csv('mnist_test.csv')

In [ ]:
df_orig_train.rename(columns={'5':'label'}, inplace=True)
df_orig_test.rename(columns={'7':'label'}, inplace=True)

In [ ]:
df_orig_train = pd.read_csv('mnist_train_final.csv')
df_orig_test = pd.read_csv('mnist_test_final.csv')
df_orig_train.head

<bound method NDFrame.head of        label  0  0.1  0.2  0.3  0.4  ...  0.612  0.613  0.614  0.615  0.616  0.617
0          0  0    0    0    0    0  ...      0      0      0      0      0      0
1          4  0    0    0    0    0  ...      0      0      0      0      0      0
2          1  0    0    0    0    0  ...      0      0      0      0      0      0
3          9  0    0    0    0    0  ...      0      0      0      0      0      0
4          2  0    0    0    0    0  ...      0      0      0      0      0      0
...      ... ..  ...  ...  ...  ...  ...    ...    ...    ...    ...    ...    ...
59994      8  0    0    0    0    0  ...      0      0      0      0      0      0
59995      3  0    0    0    0    0  ...      0      0      0      0      0      0
59996      5  0    0    0    0    0  ...      0      0      0      0      0      0
59997      6  0    0    0    0    0  ...      0      0      0      0      0      0
59998      8  0    0    0    0    0  ...      0      0   

In [ ]:
labels_train = df_orig_train['label'].to_numpy()
df_digits = df_orig_train.drop('label',axis=1)
labels_test = df_orig_test['label'].to_numpy()
df_digits_test = df_orig_test.drop('label',axis=1)
# ind = 3
# digits_data_train = df_digits.to_numpy()
# digit = digits_data_train[ind]
# digit = digit.reshape(28,28)
# plt.title('this is  --->   ' + str(labels_train[ind]))
# plt.imshow(digit, cmap='gray')

In [ ]:
df_orig_train.to_csv('mnist_train_final.csv', index=False)
df_orig_test.to_csv('mnist_test_final.csv', index=False)

PCA testing

In [ ]:
dim_red = 4
df_digits=StandardScaler().fit_transform(df_digits)
pca = PCA(n_components=dim_red, svd_solver='randomized',
          whiten=True).fit(df_digits)
data_train_pca = pca.transform(df_digits)

In [ ]:
df_digits_test=StandardScaler().fit_transform(df_digits_test)
pca = PCA(n_components=dim_red, svd_solver='randomized',
          whiten=True).fit(df_digits_test)
data_test_pca = pca.transform(df_digits_test)

In [ ]:
# param_grid = { "C" : [2]
#               , "gamma" : [2]}
# rf = svm.SVC()
# gs = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='accuracy', cv=2, n_jobs=-1, verbose=1)
# gs = gs.fit(data_train_pca, labels_train)

# print(gs.best_score_)
# print(gs.best_params_)

Fitting 2 folds for each of 1 candidates, totalling 2 fits


In [ ]:
# bp = gs.best_params_
# clf = svm.SVC(C=bp['C'], kernel='rbf', gamma=bp['gamma'])
clf = svm.SVC().fit(data_train_pca, labels_train)


In [ ]:
clf.score(data_test_pca,labels_test)

0.4955495549554956

## My code 

Importing Dataset

In [ ]:
ds = tfds.load('mnist', split='train', as_supervised=True)
dstest = tfds.load('mnist', split='test', as_supervised=True)

In [ ]:
dsinfo=tfds.builder('mnist').info
mnist_frame_train=tfds.as_dataframe(ds,dsinfo)
mnist_frame_test=tfds.as_dataframe(dstest,dsinfo)

In [ ]:
data_train = mnist_frame_train.drop(['label'], axis='columns', inplace=False)
label_train = mnist_frame_train['label']
data_test= mnist_frame_test.drop(['label'], axis='columns', inplace=False)
label_test = mnist_frame_test['label']
data_train.image.shape

(60000,)

PCA self

In [ ]:
size_data=60000
dim_number=28*28
reduced_dim=16

dataarraymnist=np.zeros((size_data,dim_number))
labelarraymnist=np.zeros((size_data,1))
count=0

for i in mnist_frame_train.image:
  dataarraymnist[count][:]=np.reshape(i,(1,28*28))
  count=count+1

counttemp=0
for i in mnist_frame_train.label:
  labelarraymnist[counttemp][:]=i
  counttemp+=1

for i in range(len(labelarraymnist)):
  labelarraymnist[i][0]=int(labelarraymnist[i][0])

labelarraymnist=labelarraymnist.flatten()

dataPCA=StandardScaler().fit_transform(dataarraymnist)

pca=PCA(n_components=reduced_dim)

dataarraymnistPCA = pca.fit_transform(dataPCA)

print(dataarraymnistPCA.shape)
print(labelarraymnist.shape)

(60000, 16)
(60000,)


In [ ]:
cl=svm.SVC(kernel='rbf',gamma='scale').fit(dataarraymnistPCA,labelarraymnist)

In [ ]:
size_test=10000
dataarraymnisttest=np.zeros((size_test,dim_number))
labelarraymnisttest=np.zeros((size_test,1))

count=0
for i in mnist_frame_test.image:
  dataarraymnisttest[count][:]=np.reshape(i,(1,28*28))
  count=count+1

counttemp=0
for i in mnist_frame_test.label:
  labelarraymnisttest[counttemp][:]=i
  counttemp+=1

for i in range(len(labelarraymnisttest)):
  labelarraymnisttest[i][0]=int(labelarraymnisttest[i][0])

labelarraymnisttest=labelarraymnisttest.flatten()

dataPCAtest=StandardScaler().fit_transform(dataarraymnisttest)

pca=PCA(n_components=reduced_dim)

dataarraymnistPCAtest = pca.fit_transform(dataPCAtest)

print(dataarraymnistPCAtest.shape)

(10000, 16)


In [ ]:
predictedlabels=cl.predict(dataarraymnistPCAtest)

In [ ]:
count=0;

for i in range(len(predictedlabels)):
  if predictedlabels[i]!=labelarraymnisttest[i]:
    count+=1

print(count*100/len(predictedlabels))

print(predictedlabels)
print(labelarraymnisttest)

50.88
[2. 0. 4. ... 8. 0. 8.]
[2. 0. 4. ... 8. 0. 5.]


#TRYING AGAIN

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [ ]:
data_train=pd.read_csv('mnist_train.csv',header=None)
data_test=pd.read_csv('mnist_test.csv',header=None)

In [ ]:
size_train=60000
size_test=10000

In [ ]:
label_train=data_train.iloc[:, [0]]
data_train=data_train.drop(data_train.columns[0],axis=1)
label_test=data_test.iloc[:, [0]]
data_test=data_test.drop(data_train.columns[0],axis=1)
# label_train
# data_train

In [ ]:
data_train=data_train.to_numpy()
label_train=label_train.to_numpy()
data_test=data_test.to_numpy()
label_test=label_test.to_numpy()

PCA

In [ ]:
dim_redn = 4
pca = PCA(n_components=dim_redn, svd_solver='randomized',
          whiten=True).fit(data_train)
data_train_pca = pca.transform(data_train)
pca = PCA(n_components=dim_redn, svd_solver='randomized',
          whiten=True).fit(data_test)
data_test_pca = pca.transform(data_test)

SVM

In [ ]:
label_train=label_train.flatten()
modelSVM=svm.SVC(gamma='scale').fit(data_train,label_train)
modelSVM.score(data_test,label_test)

#AGANE


In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import SVC

In [2]:
from keras.datasets import mnist
(data_train,label_train),(data_test,label_test)=mnist.load_data()
data_train= data_train.reshape(data_train.shape[0], -1)
data_test= data_test.reshape(data_test.shape[0], -1)

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
data_train=data_train/255.0
data_test=data_test/255.0

In [4]:
from sklearn.preprocessing import scale
data_train = scale(data_train)
data_test=scale(data_test)
data_train=data_train.tolist()
data_test=data_test.tolist()

In [5]:
dim_redn=2
pca=PCA(n_components=dim_redn)
data_PCA_train=pca.fit_transform(data_train)
data_PCA_test=pca.fit_transform(data_test)

In [6]:
dim_redn_LDA=9
lda=LDA(n_components=dim_redn_LDA)
data_LDA_train=lda.fit_transform(data_train,label_train)
data_LDA_test=lda.fit_transform(data_test,label_test)

Without Dimn Redn

In [ ]:
model_svm_linear=SVC(kernel='linear').fit(data_train,label_train)
model_svm_linear.score(data_test,label_test)

0.928

In [ ]:
model_svm_linear.score(data_train,label_train )

0.9821

With PCA

In [ ]:
model_svm_linear=SVC(kernel='linear').fit(data_PCA_train,label_train)
model_svm_linear.score(data_PCA_test,label_test)

With LDA

In [ ]:
model_svm_linear=SVC(kernel='linear').fit(data_LDA_train,label_train)
model_svm_linear.score(data_LDA_test,label_test)